## Assignment 2  

R12723006  財金碩一 林姝延


In [2]:
import statistics
import numpy as np
import scipy.stats as stats
import pandas as pd

### Black-Scholes 

In [29]:
def N(x):
    return stats.norm.cdf(x)
def ln(x):
    return np.log(x)
def e(x):
    return np.exp(x)
def norm(x):
    return np.random.normal(x)

S0 = 50
K = 50
r = 0.1
q = 0.05
sigma = 0.4
T = 0.5

d1 = (ln(S0/K)+(r-q+0.5*sigma**2)*T) / (sigma * T**0.5)
d2 = d1 - sigma * (T**0.5)

C = S0 * e(-q*T) * N(d1) - K * e(-r*T) * N(d2)
print(f"call price: {round(C,4)}")
P = K * e(-r*T) * N(-d2) - S0 * e(-q*T) * N(-d1)
print(f"put price: {round(P,4)}")

call price: 6.0396
put price: 4.8356


### MonteCarlo

In [41]:
C_payoff_list = []
P_payoff_list = []
C_option_price_list = []
P_option_price_list = []

number_of_simulations = 10000
number_of_repetitions = 20

for j in range(number_of_repetitions):
    for i in range(number_of_simulations):
        nd = []
        nd = np.random.normal()
        ST = e(ln(S0)+(r-q-0.5*(sigma**2))*T + T**0.5 * nd * sigma)
        if K < ST:
            C = e(-r*T) * (ST - K)
            P = 0
        elif K > ST:
            C = 0
            P = e(-r*T) * (K - ST)
        else:
            C = P = 0
        C_payoff_list.append(C) 
        P_payoff_list.append(P)
    C_price = statistics.mean(C_payoff_list)  
    P_price = statistics.mean(P_payoff_list)
    C_option_price_list.append(C_price)  
    P_option_price_list.append(P_price)

C_mean = np.mean(C_option_price_list)
C_sd = np.std(C_option_price_list)
#C_sd = statistics.stdev(C_option_price_list)
P_mean = statistics.mean(P_option_price_list)
P_sd = statistics.stdev(P_option_price_list)

C_low = C_mean - 2 * C_sd
C_up = C_mean + 2 * C_sd
P_low = P_mean - 2 * P_sd
P_up = P_mean + 2 * P_sd

print(f"C_mean: {round(C_mean,4)}")
print(f"C_standard deviation: {round(C_sd,4)}")
print(f"95% confidence interval: [{round(C_low,4)}, {round(C_up,4)}]")
print()
print(f"P_mean: {round(P_mean,4)}")
print(f"P_standard deviation: {round(P_sd,4)}")
print(f"95% confidence interval: [{round(P_low,4)}, {round(P_up,4)}]")

C_mean: 6.0989
C_standard deviation: 0.0519
95% confidence interval: [5.9952, 6.2026]

P_mean: 4.8517
P_standard deviation: 0.0241
95% confidence interval: [4.8034, 4.8999]


### CRR

In [35]:
def CRR_binomial_tree_model(S0, K, r, q, sigma, T, n, callorput, option_type):
    dt = T / n
    u = np.exp(sigma * dt ** 0.5)
    d = 1 / u
    # risk neutral probability
    p = (np.exp((r - q) * dt) - d) / (u - d)
    # discount factor
    df = np.exp(-r * dt)

    # stock price
    S = np.zeros([n + 1, n + 1])
    # option value
    v = np.zeros([n + 1, n + 1])
    for i in range(n + 1):
        for j in range(i + 1):
            S[j, i] = S0 * d ** j * u ** (i - j)
            if callorput == 'call':
                v[j, i] = max(S[j, i] - K, 0)
            elif callorput == 'put': 
                v[j, i] = max(K - S[j, i], 0)
    # print("S:")
    # print(pd.DataFrame(S))
    # print("v:")
    # print(pd.DataFrame(v))    
    # present value
    pv = np.zeros([n + 1, n + 1])
    pv[:, n] = v[:, n]
    # print("pv:")
    # print(pd.DataFrame(pv))
    # Starting from last column
    for i in np.arange(n - 1, -1, -1):
        for j in np.arange(0, i + 1):
            if option_type == 'European':  
                pv[j, i] = df * (p * pv[j, i + 1] + (1 - p) * pv[j + 1, i + 1])
            elif option_type == 'American': 
                pv[j, i] = max(df * (p * pv[j, i + 1] + (1 - p) * pv[j + 1, i + 1]), v[j, i])
    # print("pd_pv:")
    # print(pd.DataFrame(pv))
    return pv[0, 0]

S0 = 50
K = 50
r = 0.1
q = 0.05
sigma = 0.4
T = 0.5

# print("* European_call:")
European_call = CRR_binomial_tree_model(S0=S0, K=K, r=r, q=q, sigma=sigma, T=T, n=100, callorput='call', option_type='European')
# print("===========================================")
# print("* European_put:")
European_put = CRR_binomial_tree_model(S0=S0, K=K, r=r, q=q, sigma=sigma, T=T, n=100, callorput='put', option_type='European')
# print("===========================================")
# print("* American_call:")
American_call = CRR_binomial_tree_model(S0=S0, K=K, r=r, q=q, sigma=sigma, T=T, n=100, callorput='call', option_type='American')
# print("===========================================")
# print("* American_put:")
American_put = CRR_binomial_tree_model(S0=S0, K=K, r=r, q=q, sigma=sigma, T=T, n=100, callorput='put', option_type='American')


print(f"European call: {round(European_call,4)}")
print(f"European put: {round(European_put,4)}")
print(f"American call: {round(American_call,4)}")
print(f"American put: {round(American_put,4)}")

European call: 6.026
European put: 4.822
American call: 6.0262
American put: 4.9795


In [28]:
def CRR_binomial_tree_model(S0, K, r, q, sigma, T, n, callorput, option_type):
    dt = T / n
    u = np.exp(sigma * dt ** 0.5)
    d = 1 / u
    # risk neutral probability
    p = (np.exp((r - q) * dt) - d) / (u - d)
    # discount factor
    df = np.exp(-r * dt)

    # stock price
    S = np.zeros([n + 1, n + 1])
    # option value
    v = np.zeros([n + 1, n + 1])
    for i in range(n + 1):
        for j in range(i + 1):
            S[j, i] = S0 * d ** j * u ** (i - j)
            if callorput == 'call':
                v[j, i] = max(S[j, i] - K, 0)
            elif callorput == 'put': 
                v[j, i] = max(K - S[j, i], 0)
    # print("S:")
    # print(pd.DataFrame(S))
    # print("v:")
    # print(pd.DataFrame(v))    
    # present value
    pv = np.zeros([n + 1, n + 1])
    pv[:, n] = v[:, n]
    # print("pv:")
    # print(pd.DataFrame(pv))
    # Starting from last column
    for i in np.arange(n - 1, -1, -1):
        for j in np.arange(0, i + 1):
            if option_type == 'European':  
                pv[j, i] = df * (p * pv[j, i + 1] + (1 - p) * pv[j + 1, i + 1])
            elif option_type == 'American': 
                pv[j, i] = max(df * (p * pv[j, i + 1] + (1 - p) * pv[j + 1, i + 1]), v[j, i])
    # print("pd_pv:")
    # print(pd.DataFrame(pv))
    return pv[0, 0]

S0 = 50
K = 50
r = 0.1
q = 0.05
sigma = 0.4
T = 0.5

# print("* European_call:")
European_call = CRR_binomial_tree_model(S0=S0, K=K, r=r, q=q, sigma=sigma, T=T, n=500, callorput='call', option_type='European')
# print("===========================================")
# print("* European_put:")
European_put = CRR_binomial_tree_model(S0=S0, K=K, r=r, q=q, sigma=sigma, T=T, n=500, callorput='put', option_type='European')
# print("===========================================")
# print("* American_call:")
American_call = CRR_binomial_tree_model(S0=S0, K=K, r=r, q=q, sigma=sigma, T=T, n=500, callorput='call', option_type='American')
# print("===========================================")
# print("* American_put:")
American_put = CRR_binomial_tree_model(S0=S0, K=K, r=r, q=q, sigma=sigma, T=T, n=500, callorput='put', option_type='American')


print(f"European call: {round(European_call,4)}")
print(f"European put: {round(European_put,4)}")
print(f"American call: {round(American_call,4)}")
print(f"American put: {round(American_put,4)}")

European call: 6.0369
European put: 4.8329
American call: 6.0371
American put: 4.9859


### CRR_Step by Step

In [21]:
def CRR_binomial_tree_model(S0, K, r, q, sigma, T, n, callorput, option_type):
    dt = T / n
    u = np.exp(sigma * dt ** 0.5)
    d = 1 / u
    # risk neutral probability
    p = (np.exp((r - q) * dt) - d) / (u - d)
    # discount factor
    df = np.exp(-r * dt)

    # stock price
    S = np.zeros([n + 1, n + 1])
    # option value
    v = np.zeros([n + 1, n + 1])
    for i in range(n + 1):
        for j in range(i + 1):
            S[j, i] = S0 * d ** j * u ** (i - j)
            if callorput == 'call':
                v[j, i] = max(S[j, i] - K, 0)
            elif callorput == 'put': 
                v[j, i] = max(K - S[j, i], 0)
    print("S:")
    print(pd.DataFrame(S))
    print("v:")
    print(pd.DataFrame(v))    
    # present value
    pv = np.zeros([n + 1, n + 1])
    pv[:, n] = v[:, n]
    print("pv:")
    print(pd.DataFrame(pv))
    # Starting from last column
    for i in np.arange(n - 1, -1, -1):
        for j in np.arange(0, i + 1):
            if option_type == 'European':  
                pv[j, i] = df * (p * pv[j, i + 1] + (1 - p) * pv[j + 1, i + 1])
            elif option_type == 'American': 
                pv[j, i] = max(df * (p * pv[j, i + 1] + (1 - p) * pv[j + 1, i + 1]), v[j, i])
    print("pd_pv:")
    print(pd.DataFrame(pv))
    return pv[0, 0]

print("* European_call:")
European_call = CRR_binomial_tree_model(S0=50, K=50, r=0.1, q=0.05, sigma=0.4, T=0.5, n=4, callorput='call', option_type='European')
print("===========================================")
print("* European_put:")
European_put = CRR_binomial_tree_model(S0=50, K=50, r=0.1, q=0.05, sigma=0.4, T=0.5, n=4, callorput='put', option_type='European')
print("===========================================")
print("* American_call:")
American_call = CRR_binomial_tree_model(S0=50, K=50, r=0.1, q=0.05, sigma=0.4, T=0.5, n=4, callorput='call', option_type='American')
print("===========================================")
print("* American_put:")
American_put = CRR_binomial_tree_model(S0=50, K=50, r=0.1, q=0.05, sigma=0.4, T=0.5, n=4, callorput='put', option_type='American')


print(f"European call: {round(European_call,4)}")
print(f"European put: {round(European_put,4)}")
print(f"American call: {round(American_call,4)}")
print(f"American put: {round(American_put,4)}")

* European_call:
S:
      0          1          2          3          4
0  50.0  57.595496  66.344822  76.423258  88.032708
1   0.0  43.406172  50.000000  57.595496  66.344822
2   0.0   0.000000  37.681916  43.406172  50.000000
3   0.0   0.000000   0.000000  32.712555  37.681916
4   0.0   0.000000   0.000000   0.000000  28.398536
v:
     0         1          2          3          4
0  0.0  7.595496  16.344822  26.423258  38.032708
1  0.0  0.000000   0.000000   7.595496  16.344822
2  0.0  0.000000   0.000000   0.000000   0.000000
3  0.0  0.000000   0.000000   0.000000   0.000000
4  0.0  0.000000   0.000000   0.000000   0.000000
pv:
     0    1    2    3          4
0  0.0  0.0  0.0  0.0  38.032708
1  0.0  0.0  0.0  0.0  16.344822
2  0.0  0.0  0.0  0.0   0.000000
3  0.0  0.0  0.0  0.0   0.000000
4  0.0  0.0  0.0  0.0   0.000000
pd_pv:
          0         1          2          3          4
0  5.713334  9.969635  16.755178  26.568212  38.032708
1  0.000000  1.816081   3.777608   7.857756  1